In [2]:
import csv
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
url_xls = "https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide"
url_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
url_countries_csv = "https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv"

In [4]:
page = requests.get(url_xls).text

In [5]:
soup = BeautifulSoup(page, 'html.parser')

In [6]:
data_attribute = soup.find_all("a", attrs={"data-toggle": "tooltip"})

In [7]:
for link in data_attribute:
    data_link = link.get('href')

In [8]:
with requests.Session() as s:
    download = s.get(data_link).content
    dataframe = pd.read_excel(download)

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'DateRep,'

In [ ]:
dataframe.head()

In [9]:
dataframe.groupby("Countries and territories").mean().head()

NameError: name 'dataframe' is not defined

In [10]:
df_github = pd.read_csv(url_csv)

In [11]:
data_by_country = df_github.groupby("Country/Region").mean()

In [12]:
df_github.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0


In [13]:
comparison = df_github.columns[0:4]==pd.Index(['Province/State', 'Country/Region', 'Lat', 'Long'])
comparison.all()

True

In [14]:
df_github_countries = df_github.iloc[:, 0:4]

In [15]:
df_github_countries

,Province/State,Country/Region,Lat,Long
0,NaN,Thailand,15.0000,101.0000
1,NaN,Japan,36.0000,138.0000
2,NaN,Singapore,1.2833,103.8333
3,NaN,Nepal,28.1667,84.2500
4,NaN,Malaysia,2.5000,112.5000
...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100
497,NaN,Puerto Rico,18.2000,-66.5000
498,NaN,Republic of the Congo,-1.4400,15.5560
499,NaN,The Bahamas,24.2500,-76.0000


In [16]:
df_github_countries.iloc[0,0]

nan

In [17]:
country_df = df_github_countries.iloc[:, 1:4].drop_duplicates(subset=['Country/Region'],keep='last',ignore_index=False)

In [18]:
len(country_df)

183

In [19]:
country_df.to_csv("data/reference/country.csv")

In [20]:
continents = ["Africa", "Antarctica", "Asia", "Europe", "North America", "Oceania", "South America", "None"]

In [262]:
continents_df = pd.DataFrame(continents)

In [263]:
continents_df.to_csv("data/reference/continent.csv")

In [264]:
prov_state = df_github.iloc[:, 0].drop_duplicates(keep='first')

In [265]:
prov_state = prov_state.reset_index()

In [266]:
prov_state = prov_state.drop(["index"],axis=1)

In [267]:
prov_state.iloc[0, :] = "No province/state"

In [268]:
prov_state

,Province/State
0,No province/state
1,British Columbia
2,New South Wales
3,Victoria
4,Queensland
...,...
321,Sint Maarten
322,Isle of Man
323,Northwest Territories
324,United States Virgin Islands


In [269]:
prov_state.to_csv("data/reference/province_state.csv")

In [270]:
import math
region_codes = []
for region in df_github_countries.iloc[:,0]:
    if isinstance(region, str):
        pass
    elif np.isnan(region):
        region_codes.append(322)
        continue
    for i, item in enumerate(prov_state.iloc[:,0]):
        if item == region:
            region_codes.append(i)

print(len(region_codes))

501


In [271]:
prov_state.iloc[:,0]

0                 No province/state
1                  British Columbia
2                   New South Wales
3                          Victoria
4                        Queensland
                   ...             
321                    Sint Maarten
322                     Isle of Man
323           Northwest Territories
324    United States Virgin Islands
325                              US
Name: Province/State, Length: 326, dtype: object

In [272]:
for i, item in enumerate(prov_state.iloc[:,0]):
    print(item)

No province/state
British Columbia
New South Wales
Victoria
Queensland
South Australia
From Diamond Princess
Western Australia
Tasmania
Northern Territory
Ontario
Alberta
Quebec
Washington
New York
California
Massachusetts
Diamond Princess
Grand Princess
Georgia
Colorado
Florida
New Jersey
Oregon
Texas
Illinois
Pennsylvania
Iowa
Maryland
North Carolina
South Carolina
Tennessee
Virginia
Arizona
Indiana
Kentucky
District of Columbia
Nevada
New Hampshire
Minnesota
Nebraska
Ohio
Rhode Island
Wisconsin
Connecticut
Hawaii
Oklahoma
Utah
Kansas
Louisiana
Missouri
Vermont
Alaska
Arkansas
Delaware
Idaho
Maine
Michigan
Mississippi
Montana
New Mexico
North Dakota
South Dakota
West Virginia
Wyoming
Hubei
France
Guangdong
Henan
Zhejiang
Hunan
Anhui
Jiangxi
Shandong
Jiangsu
Chongqing
Sichuan
Heilongjiang
Denmark
Beijing
Shanghai
Hebei
Fujian
Guangxi
Shaanxi
Yunnan
Hainan
Guizhou
Tianjin
Shanxi
Gansu
Hong Kong
Liaoning
Jilin
Xinjiang
Inner Mongolia
Ningxia
Qinghai
Macau
Faroe Islands
St Martin
Channel

In [273]:
df_github_countries["region_codes"] = region_codes

In [274]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100,322,Europe,3
497,NaN,Puerto Rico,18.2000,-66.5000,322,North America,4
498,NaN,Republic of the Congo,-1.4400,15.5560,322,Africa,0
499,NaN,The Bahamas,24.2500,-76.0000,322,North America,4


In [275]:
df_countries_continents = pd.read_csv(url_countries_csv)

In [276]:
new_country_data = {
    "Continent": [
        "Europe",
        "North America",
        "Africa",
        "Europe",
        "None",
        "Europe",
        "Asia",
        "Asia",
        "Africa",
        "Africa",
        "Africa",
        "Europe",
        "Africa",
        "Africa",
        "North America",
        "Africa",
        "Asia",
        "North America",
        "Africa",
        "South America",
        "Africa",
        "North America",
        "Oceania",
        "Europe",
        "Europe",
        "North America",
        "Africa",
        "North America",
        "Africa",
                 ],
    "Country": [
        "North Macedonia",
        "Martinique",
        "Burkina Faso",
        "Holy See",
        "Cruise Ship",
        "Czechia",
        "Taiwan*",
        "Russia",
        "Congo (Kinshasa)",
        "Cote d'Ivoire",
        "Eswatini",
        "Kosovo",
        "Congo (Brazzaville)",
        "Gambia, The",
        "Bahamas, The",
        "Cabo Verde",
        "Timor-Leste",
        "Guadeloupe",
        "Reunion",
        "French Guiana",
        "Mayotte",
        "Greenland",
        "Guam",
        "Guernsey",
        "Jersey",
        "Puerto Rico",
        "Republic of the Congo",
        "The Bahamas",
        "The Gambia"
    ]
}
new_country_data_df = pd.DataFrame(new_country_data)

In [277]:
df_countries_continents=df_countries_continents.append(new_country_data_df, ignore_index=True)

In [278]:
df_countries_continents.to_csv("data/reference/countries_continents.csv")

In [279]:
df_countries_continents

,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
218,Europe,Jersey
219,North America,Puerto Rico
220,Africa,Republic of the Congo
221,North America,The Bahamas


In [280]:
continent_codes = []
for item in df_github_countries.iloc[:,1]:
    old_size = len(continent_codes)
    for country in df_countries_continents.iterrows():
        if item==country[1][1]:
            continent_codes.append(country[1][0])
            
    new_size = len(continent_codes)
    if old_size==new_size:
        continent_codes.append("NA")

In [281]:
df_github_countries['continent'] = continent_codes

In [282]:
for i, line in enumerate(df_github_countries.iterrows()):
    if line[1][5]=="NA":
        print(line)

In [283]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100,322,Europe,3
497,NaN,Puerto Rico,18.2000,-66.5000,322,North America,4
498,NaN,Republic of the Congo,-1.4400,15.5560,322,Africa,0
499,NaN,The Bahamas,24.2500,-76.0000,322,North America,4


In [284]:
import math
continent_codes = []
for continent in df_github_countries.iloc[:,-1]:
    if isinstance(continent, str):
        pass
    elif np.isnan(continent):
        print("WHAT")
    for i, item in enumerate(continents_df.iloc[:,0]):
        if item == continent:
            continent_codes.append(i)

print(len(continent_codes))

0


In [137]:
continents_df_csv = pd.read_csv("data/reference/continent.csv", index_col=0)
continents_df_csv

,0
0,Africa
1,Antarctica
2,Asia
3,Europe
4,North America
5,Oceania
6,South America
7,None


In [78]:
df_github_countries['continent_codes'] = continent_codes

In [285]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100,322,Europe,3
497,NaN,Puerto Rico,18.2000,-66.5000,322,North America,4
498,NaN,Republic of the Congo,-1.4400,15.5560,322,Africa,0
499,NaN,The Bahamas,24.2500,-76.0000,322,North America,4


In [286]:
cases = df_github_countries.drop("continent", axis=1).drop("Lat", axis=1).drop("Long", axis=1).drop("Province/State", axis=1)

In [287]:
cases

,Country/Region,region_codes,continent_codes
0,Thailand,322,2
1,Japan,322,2
2,Singapore,322,2
3,Nepal,322,2
4,Malaysia,322,2
...,...,...,...
496,Jersey,322,3
497,Puerto Rico,322,4
498,Republic of the Congo,322,0
499,The Bahamas,322,4


In [288]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
496,NaN,Jersey,49.1900,-2.1100,322,Europe,3
497,NaN,Puerto Rico,18.2000,-66.5000,322,North America,4
498,NaN,Republic of the Congo,-1.4400,15.5560,322,Africa,0
499,NaN,The Bahamas,24.2500,-76.0000,322,North America,4


In [289]:
import math
country_codes = []
country_df = pd.read_csv("data/reference/country.csv", index_col=0)

for case in df_github_countries.iloc[:,1]:
    for i, item in enumerate(country_df.iloc[:,0]):
        if item == case:
            country_codes.append(i)
            
print(len(country_codes))

501


In [290]:
country_df

,Country/Region,Lat,Long
0,Thailand,15.0000,101.0000
1,Japan,36.0000,138.0000
2,Singapore,1.2833,103.8333
3,Nepal,28.1667,84.2500
4,Malaysia,2.5000,112.5000
...,...,...,...
496,Jersey,49.1900,-2.1100
497,Puerto Rico,18.2000,-66.5000
498,Republic of the Congo,-1.4400,15.5560
499,The Bahamas,24.2500,-76.0000


In [291]:
cases['country_code'] = country_codes

In [292]:
cases = cases.drop('Country/Region', axis=1)

In [293]:
cases

,region_codes,continent_codes,country_code
0,322,2,0
1,322,2,1
2,322,2,2
3,322,2,3
4,322,2,4
...,...,...,...
496,322,3,178
497,322,4,179
498,322,0,180
499,322,4,181


In [294]:
cases_full = pd.concat([cases, df_github.iloc[:,4:]], axis=1)

In [295]:
cases_full

,region_codes,continent_codes,country_code,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,322,2,0,2,3,5,7,8,8,14,...,82,114,147,177,212,272,322,411,599,599.0
1,322,2,1,2,1,2,2,4,4,7,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,322,2,2,0,1,3,3,4,5,7,...,212,226,243,266,313,345,385,432,455,455.0
3,322,2,3,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,322,2,4,0,0,0,3,4,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,322,3,178,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
497,322,4,179,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
498,322,0,180,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
499,322,4,181,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [296]:
cases_full_melt = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=['1/22/20'])

In [297]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2
1,322,2,1,1/22/20,2
2,322,2,2,1/22/20,0
3,322,2,3,1/22/20,0
4,322,2,4,1/22/20,0
...,...,...,...,...,...
496,322,3,178,1/22/20,0
497,322,4,179,1/22/20,0
498,322,0,180,1/22/20,0
499,322,4,181,1/22/20,0


In [298]:
cases_full.shape

(501, 65)

In [299]:
for i, column in enumerate(cases_full.iloc[:,4:]):
    melted_df = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=[column])
    cases_full_melt = cases_full_melt.append(melted_df)

In [300]:
cases_full_melt.shape

(31062, 5)

In [92]:
cases_full_melt = cases_full_melt.reset_index()

In [93]:
cases_full_melt = cases_full_melt.drop("index", axis=1)

In [95]:
cases_full_melt.head()

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2.0
1,322,2,1,1/22/20,2.0
2,322,2,2,1/22/20,0.0
3,322,2,3,1/22/20,0.0
4,322,2,4,1/22/20,0.0


In [96]:
cases_full_melt['variable'] = pd.to_datetime(cases_full_melt['variable'],infer_datetime_format=True)

In [98]:
cases_full_melt.head()

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,2020-01-22,2.0
1,322,2,1,2020-01-22,2.0
2,322,2,2,2020-01-22,0.0
3,322,2,3,2020-01-22,0.0
4,322,2,4,2020-01-22,0.0


In [99]:
cases_full_melt.to_csv('data/timeseries/cases_confirmed.csv')

In [1]:
sources = {
    "confirmed": {
        "url": 
    },
    
}

In [2]:
sources['type']

'confirmed'

In [27]:
sources = {
    "confirmed": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",
        "local_path": "data/cases_confirmed.csv"
    },
    "recovered": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv",
        "local_path": "data/cases_recovered.csv"    
    },
    "deaths": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",
        "local_path": "data/cases_deaths.csv"
    },
}

In [38]:
for key, value in sources.items():
    print(value['url'])

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv


In [11]:
sources['confirmed']['url']

'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

In [15]:
import pandas as pd

In [22]:
def checkDataChange(data_type):
    old_data = pd.read_csv(sources[data_type]['local_path'])
    old_size = old_data.size
    
    new_data = pd.read_csv(sources[data_type]['url'])
    new_size = new_data.size
    if old_size!=new_size:
        print("Data size mismatch: Old: ",  str(old_size), "; New: ", str(new_size))

In [23]:
checkDataChange('confirmed')

Data size mismatch: Old:  178242 ; New:  33066


In [414]:
class CovidDatasetTest:
    def __init__(self):
        self.sources = {
            "confirmed": {
                "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",
                "local_path": "data/timeseries/cases_confirmed.csv"
            },
            "recovered": {
                "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv",
                "local_path": "data/timeseries/cases_recovered.csv"    
            },
            "deaths": {
                "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",
                "local_path": "data/timeseries/cases_deaths.csv"
            },
        }
        self.confirmed_data_raw = {
            'new': self.getNewData('confirmed'),
            'old': self.getOldData('confirmed')
            }
        self.recovered_data_raw = {
            'new': self.getNewData('recovered'),
            'old': self.getOldData('recovered')
            }
        self.deaths_data_raw = {
            'new': self.getNewData('deaths'),
            'old': self.getOldData('deaths')
            }
        self.full_dataset_raw = [self.confirmed_data_raw, self.recovered_data_raw, self.deaths_data_raw]

        self.reference_data = self.makeReferenceData()

    def getNewData(self, data_type):
        """Input url of csv, returns dataframe"""
        df = pd.read_csv(self.sources[str(data_type)]['url'])
        return df

    def getOldData(self, data_type):
        """Input filepath of csv, returns dataframe"""
        df = pd.read_csv(self.sources[str(data_type)]['local_path'])
        return df

    def makeReferenceData(self):
        continents = ["Africa", "Antarctica", "Asia", "Europe", "North America", "Oceania", "South America", "None"]
        continents_df = pd.DataFrame(continents)
        return continents_df

    def prepData(self):
        full_dataset_cleaned = []
        for i, dataset in enumerate(self.full_dataset_raw):
            countries = dataset['new'].iloc[:,:4]
            timeseries = dataset['new'].iloc[:,4:]
#             print(i)
            # Create list of region codes present in the dataset and add column
            region_codes = []
            prov_state = countries.iloc[:, 0].drop_duplicates(keep='first')
            prov_state = prov_state.reset_index()
            prov_state = prov_state.drop(["index"],axis=1)
            prov_state.iloc[0, :] = "No province/state"
            for region in countries.iloc[:,0]:
                if isinstance(region, str):
                    pass
                elif np.isnan(region):
                    region_codes.append(0)
                    continue
                for i, item in enumerate(prov_state.iloc[:,0]):
                    if item == region:
                        region_codes.append(i)
            countries["region_codes"] = region_codes

            # Create list of continent codes and add column
            continents = []
            df_countries_continents = pd.read_csv("data/reference/countries_continents.csv", index_col=0)
            for item in countries.iloc[:,1]:
                old_size = len(continents)
                for country in df_countries_continents.iterrows():
                    if item==country[1][1]:
                        continents.append(country[1][0])
            
                new_size = len(continents)
                if old_size==new_size:
                    continents.append("NA")
            for i, line in enumerate(countries.iterrows()):
                if line[1][-1]=="NA":
                    print(line)
            countries["continent"] = continents
            
            continent_codes = []
            continents_df = pd.read_csv("data/reference/continent.csv", index_col=0)
            for continent in countries.iloc[:,-1]:
                if isinstance(continent, str):
                    pass
                elif np.isnan(continent):
                    print("Not a continent")
                for i, item in enumerate(continents_df.iloc[:,0]):
                    if item == continent:
                        continent_codes.append(i)
            countries["continent_codes"] = continent_codes

            # Create and append list of country codes
            country_codes = []
            country_df = pd.read_csv("data/reference/country.csv", index_col=0)
            for case in countries.iloc[:,1]:
                for i, item in enumerate(country_df.iloc[:,0]):
                    if item == case:
                        country_codes.append(i)
            countries['country_code'] = country_codes
            
            # Concatenate country and timeseries dataframes       
            cases = countries.drop('Country/Region', axis=1).drop("continent", axis=1).drop("Lat", axis=1).drop("Long", axis=1).drop("Province/State", axis=1)
            cases_full = pd.concat([cases, timeseries], axis=1)
            cases_full_melt = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=['1/22/20'])
            
            # Join all melted time series columns
            for i, column in enumerate(cases_full.iloc[:,4:]):
                melted_df = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=[column])
                cases_full_melt = cases_full_melt.append(melted_df)
            cases_full_melt['variable'] = pd.to_datetime(cases_full_melt['variable'],infer_datetime_format=True)
            cases_full_melt.columns = ['region_code', 'continent_code', 'country_code', 'date', 'count']
#             cases_full_melt.astype({'count': 'int32',}).dtypes
            
            full_dataset_cleaned.append(cases_full_melt)
        self.full_dataset_cleaned = full_dataset_cleaned
    
    def combineCleansedData(self):
        full_dataset_combined = pd.DataFrame()
        case_types = ["Confirmed", "Recovered", "Deaths"]
        case_types_df = pd.DataFrame(case_types)
        for i, dataset in enumerate(self.full_dataset_cleaned):
            dataset['case_type'] = i
            full_dataset_combined = full_dataset_combined.append(dataset, ignore_index=True)
        self.full_dataset_combined = full_dataset_combined
    
    def saveData(self):
        filenames = ["confirmed", "recovered", "deaths"]
        base_dir_split = "data/timeseries/daily_split/"
        base_dir_combined = "data/timeseries/daily_combined/"
        for i, item in enumerate(self.full_dataset_cleaned):
            filename = base_dir_split + str(date.today()) + str(filenames[i]) + ".csv" 
            item.to_csv(filename)
        filename_combined = base_dir_combined + str(date.today()) + "-combined.csv" 
        self.full_dataset_combined.to_csv(filename_combined)

In [415]:
cases = CovidDatasetTest()

In [416]:
cases.sources

{'confirmed': {'url': 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv',
  'local_path': 'data/timeseries/cases_confirmed.csv'},
 'recovered': {'url': 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv',
  'local_path': 'data/timeseries/cases_recovered.csv'},
 'deaths': {'url': 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv',
  'local_path': 'data/timeseries/cases_deaths.csv'}}

In [417]:
cases.prepData()

In [418]:
cases.combineCleansedData()

In [419]:
cases.full_dataset_combined

,region_code,continent_code,country_code,date,count,case_type
0,0,2,0,2020-01-22,2.0,0
1,0,2,1,2020-01-22,2.0,0
2,0,2,2,2020-01-22,0.0,0
3,0,2,3,2020-01-22,0.0,0
4,0,2,4,2020-01-22,0.0,0
...,...,...,...,...,...,...
93181,0,3,178,2020-03-23,0.0,2
93182,0,4,179,2020-03-23,1.0,2
93183,0,0,180,2020-03-23,0.0,2
93184,0,4,181,2020-03-23,0.0,2


In [420]:
cases.saveData()

In [396]:
cases.full_dataset_cleaned[0]['case_type'] = 0

In [397]:
cases.full_dataset_cleaned[0]

,region_code,continent_code,country_code,date,count,case_type
0,0,2,0,2020-01-22,2.0,0
1,0,2,1,2020-01-22,2.0,0
2,0,2,2,2020-01-22,0.0,0
3,0,2,3,2020-01-22,0.0,0
4,0,2,4,2020-01-22,0.0,0
...,...,...,...,...,...,...
496,0,3,178,2020-03-23,0.0,0
497,0,4,179,2020-03-23,0.0,0
498,0,0,180,2020-03-23,0.0,0
499,0,4,181,2020-03-23,0.0,0


In [392]:
types = ["Confirmed", "Recovered", "Deaths"]
types_df = pd.DataFrame(types)
types_df.to_csv("data/reference/types.csv")

In [390]:
np.issubdtype(cases.full_dataset_cleaned[2]['count'].dtype, np.number)

True